In [1]:
import shutil
import os
import pandas as pd
import audio2numpy as a2n
from scipy.io import wavfile
from pydub import AudioSegment, effects
import numpy as np

# Obtaining Sections and Segment Split Points

In [2]:
loc = '../saraga/dataset/carnatic/'
sections_main = {}
sections = {}
pallavi_start = {}
pallavi_start_main = {}
violin_alap_start = {}
violin_alap_start_main = {}
count = 0
sec_present_count = 0

In [3]:
for sec_loc in os.scandir(loc):
    if sec_loc.is_file():
        continue
    for filename in os.scandir(sec_loc.path):
        if filename.is_file():
            continue
        for filename2 in os.scandir(filename.path):
            if(filename2.path.endswith('sections-manual-p.txt')):
                sec_present_count = 1
                section_data = pd.read_csv(filename2, sep = '\t',names=['Start','Number','Time','Section'])
                sections.update({filename.name:section_data})

                for i in range(len(section_data['Section'])):
                    if(section_data['Section'][i]=='Violin ālāp'):
                        violin_alap_start.update({filename.name:section_data['Start'][i]})
                        count+=1
                    elif(section_data['Section'][i]=='Pallavi'):
                        pallavi_start.update({filename.name:section_data['Start'][i]})
                        count+=1
                        break
                
                if(count != 2):
                    pallavi_start.update({filename.name:0})
                    violin_alap_start.update({filename.name:0})
                count = 0
            
        if(sec_present_count == 0):
            pallavi_start.update({filename.name:0})
            violin_alap_start.update({filename.name:0})
        sec_present_count = 0
    sections_main.update({sec_loc.name:sections})
    sections = {}
    pallavi_start_main.update({sec_loc.name:pallavi_start})
    pallavi_start = {}
    violin_alap_start_main.update({sec_loc.name:violin_alap_start})
    violin_alap_start = {}

In [4]:
sections_main["Akkarai Sisters at Arkay by Akkarai Sisters"]["Apparama Bhakti"]

,Start,Number,Time,Section
0,1.363265,1,473.705397,Vocal ālāp
1,475.068662,1,268.248526,Violin ālāp
2,743.317188,1,77.543039,Pallavi
3,820.860227,1,79.191655,Anupallavi
4,900.051882,1,87.539229,Caraṇam
5,987.591111,1,325.729524,Nereval
6,1313.320635,1,295.169161,Kalpanā svara


In [5]:
pallavi_start_main["Akkarai Sisters at Arkay by Akkarai Sisters"]["Apparama Bhakti"]

743.317188208

In [6]:
violin_alap_start_main["Akkarai Sisters at Arkay by Akkarai Sisters"]["Apparama Bhakti"]

475.068662131

In [7]:
sections_main["Sumitra Vasudev at Arkay by Sumithra Vasudev"]["Vanajaksha Ninne Kori"]

,Start,Number,Time,Section
0,0.092880,1,39.659683,Ślōka
1,39.752562,1,96.595011,Pallavi
2,136.347574,1,71.424580,Anupallavi
3,207.772154,1,137.555011,Muktāyi svara
4,345.327166,1,22.105397,Caraṇam
5,367.432562,1,149.661315,Ciṭṭa svara


In [8]:
pallavi_start_main["Sumitra Vasudev at Arkay by Sumithra Vasudev"]["Vanajaksha Ninne Kori"]

0

In [9]:
violin_alap_start_main["Sumitra Vasudev at Arkay by Sumithra Vasudev"]["Vanajaksha Ninne Kori"]

0

Reason: Violin Alap does not exist so 0

# Splitting the Audio

In [10]:
# new_loc = '/Volumes/T7/Music Dataset/carnatic/'
new_loc = '../Datasets/carnatic/' # Write the folder where the carnatic dataset exists
new_folder = 'segmented_audio'
data_folder = 'violin_solo_dataset'

In [11]:
def slug(file):
    """
    Returns slug of file
    file: the mp3 file name or path
    """
    count = 0
    loc = 0
    loc2 = 0
    s1 = file[::-1]
    for i in s1:
        if(i=='.'):
            if(count==0):
                loc2 = loc
            count+=1
        if(count == 2):
            break
        loc+=1
    return(s1[loc:loc2:-1])

In [12]:
path_1 = os.path.join(new_loc, new_folder)
print(path_1)

../Datasets/carnatic/segmented_audio


- Loudness normalization adjusts the recording based on perceived loudness. 
- Normalization differs from dynamic range compression, which applies varying levels of gain over a recording to fit the level within a minimum and maximum range. 
- Normalization adjusts the gain by a constant value across the entire recording.

In [13]:
new_path = os.path.join(new_loc, data_folder)
if(not(os.path.isdir(new_path))):
    os.mkdir(new_path) # Creates Violin_solos directory

In [16]:
for concerts in os.scandir(new_loc):
    if concerts.is_file():
        continue
    for songs in os.scandir(concerts.path):
        if songs.is_file():
            continue
        path = os.path.join(songs.path, new_folder)
        if(not(os.path.isdir(path))):
            os.mkdir(path) #Creates folder segmented_audio in every folder if already not present
        for multitracks in os.scandir(songs.path):
            # if(multitracks.name.endswith('.mp3') and not(multitracks.name.endswith('.mp3.mp3'))): # For including percussion instruments for segmentation           
            if(multitracks.name.endswith('.multitrack-violin.mp3')):
                # audio_data, fs = a2n.audio_from_file(multitracks.path)
                audio_data = AudioSegment.from_file(multitracks.path, "mp3")
                slugs = slug(multitracks.name)
                fs = int(audio_data.frame_rate)

                # cut_point = round(pallavi_start_main[concerts.name][songs.name]*fs)
                cut_point1 = violin_alap_start_main[concerts.name][songs.name] * 1000 # ms
                cut_point2 = pallavi_start_main[concerts.name][songs.name] * 1000 # ms
                if cut_point1 != 0 and cut_point2 != 0:
                    violin_solo = audio_data[cut_point1:cut_point2]
                    
                    # Peak normalizing the sounds
                    violin_solo = effects.normalize(violin_solo)

                    # wavfile.write(songs.path + '/' + new_folder + '/' + mains + '/' + songs.name + slugs + '-main.wav',fs,main)
                    # violin_solo.export(songs.path + '/' + new_folder + '/' + songs.name + slugs + '-violin_solo.wav', format = 'wav')
                    # violin_solo.export(new_loc + songs.name + slugs + '-violin_solo.wav', format = 'wav')

## To write on the violin_solos dataset

In [15]:
data_num = 0
for concerts in os.scandir(new_loc):
    if concerts.is_file():
        continue
    for songs in os.scandir(concerts.path):
        if songs.is_file():
            continue
        path = os.path.join(songs.path, new_folder)
        if(not(os.path.isdir(path))):
            os.mkdir(path) #Creates folder segmented_audio in every folder if already not present
        for multitracks in os.scandir(songs.path):
            # if(multitracks.name.endswith('.mp3') and not(multitracks.name.endswith('.mp3.mp3'))): # For including percussion instruments for segmentation           
            if(multitracks.name.endswith('.multitrack-violin.mp3')):
                # audio_data, fs = a2n.audio_from_file(multitracks.path)
                audio_data = AudioSegment.from_file(multitracks.path, "mp3")
                slugs = slug(multitracks.name)
                fs = int(audio_data.frame_rate)

                # cut_point = round(pallavi_start_main[concerts.name][songs.name]*fs)
                cut_point1 = violin_alap_start_main[concerts.name][songs.name] * 1000 # ms
                cut_point2 = pallavi_start_main[concerts.name][songs.name] * 1000 # ms
                if cut_point1 != 0 and cut_point2 != 0:
                    violin_solo = audio_data[cut_point1:cut_point2]
                    print(f'Concert: {concerts.name}')
                    print(f'Song: {songs.name}')
                    
                    # Peak normalizing the sounds
                    violin_solo = effects.normalize(violin_solo)

                    # wavfile.write(songs.path + '/' + new_folder + '/' + mains + '/' + songs.name + slugs + '-main.wav',fs,main)
                    # violin_solo.export(songs.path + '/' + new_folder + '/' + songs.name + slugs + '-violin_solo.wav', format = 'wav')
                    violin_solo.export(new_loc + data_folder + '/' + 'violin_solo' + str(data_num) + '.wav', format = 'wav')
                    data_num+=1

Concert: V Shankarnarayanan at Arkay by V. Shankaranarayanan
Song: Dinamani Vamsa
Concert: V Shankarnarayanan at Arkay by V. Shankaranarayanan
Song: Chakkani Raja
Concert: V Shankarnarayanan at Arkay by V. Shankaranarayanan
Song: Ardhanareeshwaram
Concert: Akkarai Sisters at Arkay by Akkarai Sisters
Song: Apparama Bhakti
Concert: Akkarai Sisters at Arkay by Akkarai Sisters
Song: Shankari Shankuru
Concert: Sumitra Vasudev at Arkay by Sumithra Vasudev
Song: Mati Matiki
Concert: Sumitra Vasudev at Arkay by Sumithra Vasudev
Song: Ramabhi Rama Manasu
Concert: Cherthala Ranganatha Sharma at Arkay by Cherthala Ranganatha Sharma
Song: Rama Rama Guna Seema
Concert: Cherthala Ranganatha Sharma at Arkay by Cherthala Ranganatha Sharma
Song: Prathi Vaaram Vaaram
Concert: Cherthala Ranganatha Sharma at Arkay by Cherthala Ranganatha Sharma
Song: Karuna Nidhi Illalo
Concert: Cherthala Ranganatha Sharma at Arkay by Cherthala Ranganatha Sharma
Song: Velum Mayilume
Concert: KP Nandini at Arkay by KP Nand

Janakipathe: Violin solo start: 825 ms

Manasaramathi: Violin solo start: 572 ms

In [22]:
# Rewriting the last two
last_loc = '../Datasets/carnatic/KP Nandini at Arkay by KP Nandini/'
loc1 = 'Janakipathe Jaya Karunya Jaladhe/Janakipathe Jaya Karunya Jaladhe.multitrack-violin.mp3'
loc2 = 'Manasaramathi/Manasaramathi.multitrack-violin.mp3'

In [23]:
last_loc+loc1

'../Datasets/carnatic/KP Nandini at Arkay by KP Nandini/Janakipathe Jaya Karunya Jaladhe/Janakipathe Jaya Karunya Jaladhe.multitrack-violin.mp3'

In [27]:
audio_data = AudioSegment.from_file(last_loc+loc1, "mp3")
slugs = 'multitrack-violin'
fs = 44100
cut_point1 = 820 * 1000 # ms
cut_point2 = (820+385.151) * 1000 # ms
violin_solo = audio_data[cut_point1:cut_point2]

# Peak normalizing the sounds
violin_solo = effects.normalize(violin_solo)

# wavfile.write(songs.path + '/' + new_folder + '/' + mains + '/' + songs.name + slugs + '-main.wav',fs,main)
# violin_solo.export(songs.path + '/' + new_folder + '/' + songs.name + slugs + '-violin_solo.wav', format = 'wav')
violin_solo.export(new_loc + data_folder + '/' + 'violin_solo' + str(11) + '.wav', format = 'wav')

<_io.BufferedRandom name='../Datasets/carnatic/violin_solo_dataset/violin_solo11.wav'>

In [28]:
audio_data = AudioSegment.from_file(last_loc+loc2, "mp3")
slugs = 'multitrack-violin'
fs = 44100
cut_point1 = 570 * 1000 # ms
cut_point2 = (570+305) * 1000 # ms
violin_solo = audio_data[cut_point1:cut_point2]

# Peak normalizing the sounds
violin_solo = effects.normalize(violin_solo)

# wavfile.write(songs.path + '/' + new_folder + '/' + mains + '/' + songs.name + slugs + '-main.wav',fs,main)
# violin_solo.export(songs.path + '/' + new_folder + '/' + songs.name + slugs + '-violin_solo.wav', format = 'wav')
violin_solo.export(new_loc + data_folder + '/' + 'violin_solo' + str(12) + '.wav', format = 'wav')

<_io.BufferedRandom name='../Datasets/carnatic/violin_solo_dataset/violin_solo12.wav'>

# Removing segmented_audio folder

In [11]:
# for sec_loc in os.scandir(new_loc):
#     if sec_loc.is_file():
#         continue
#     for song in os.scandir(sec_loc.path):
#         if song.is_file():
#             continue
#         path = os.path.join(song.path, new_folder)
#         if(os.path.isdir(path)):
#             shutil.rmtree(path) #Creates folder segmented_audio in every folder if already not present